In [66]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import random

import matplotlib.pyplot as plt

In [71]:
subset_size_train = 5000
subset_size_test = 500

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Descargar y cargar el dataset CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

def create_subset(dataset, subset_size):
  indices = list(range(len(dataset)))
  random.shuffle(indices)
  return torch.utils.data.Subset(dataset, indices[:subset_size])

trainset_subset = create_subset(trainset, subset_size_train)
testset_subset = create_subset(testset, subset_size_test)

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=64, shuffle=False)
testloader = torch.utils.data.DataLoader(testset_subset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [72]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=11, padding=1)
        self.maxpool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = CNN()

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [74]:
num_epochs = 8

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print('+ epoch')

print('Finished Training')

+ epoch
+ epoch
+ epoch
+ epoch
+ epoch
+ epoch
+ epoch
+ epoch
Finished Training


In [78]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# Predecir la clase de las imágenes de prueba
model.eval()
with torch.no_grad():
    outputs = model(images)
predicted = torch.argmax(outputs, 1)

# Mostrar algunas imágenes y sus predicciones
plt.figure(figsize=(8, 8))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(images[i].view(10, 10, 3))
    plt.title(f'Predicción: {predicted[i]}')
plt.tight_layout()
plt.show()

: 